In [ ]:
SYFT_VERSION = ">=0.8.1b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
%pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html -q

In [ ]:
import syft as sy
sy.requires(SYFT_VERSION)

In [ ]:
node = sy.orchestra.launch(name="test-domain_api", port="auto", dev_mode=True, reset=True)


In [ ]:
domain_client = node.login(email="info@openmined.org", password="changethis")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical, plot_model


db=pd.read_pickle("../datasets/sample.pkl")
mouse_data = db[db['type'].isin(['mousemove', 'mousedown', 'mouseup'])]
keyboard_data = db[db['type'].isin(['keyup', 'keydown'])]
keyboard_data=keyboard_data.drop(['ID', 'INVALID', 'X', 'Y', 'resolutionX', 'resolutionY', 'mu'], axis=1)
mouse_data=mouse_data.drop(['ID', 'INVALID', 'type', 'value', 'mu'], axis=1)
keystrokes = keyboard_data['value'].unique()
def load_mouse_data():
    USERS = set(mouse_data['user'])
    X_train = pd.DataFrame(columns = ['O','C','E','A','N'])
    X_test  = pd.DataFrame(columns = ['O','C','E','A','N'])
    y_train = pd.DataFrame()
    y_test  = pd.DataFrame()
    for index,user in enumerate(USERS) :
     X_user= mouse_data[mouse_data['user']==user]
     X = X_user[['O','C','E','A','N']]
     y = X_user['user']
     X_train_user, X_test_user, y_train_user, y_test_user = train_test_split(X, y, test_size=0.2, random_state=42)
     X_train = pd.concat([X_train,X_train_user])
     X_test= pd.concat([X_test,X_test_user])
     y_train = pd.concat([y_train,y_train_user])
     y_test= pd.concat([y_test,y_test_user])
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    return  np.asarray(X_train).astype('float32'), np.asarray(X_test).astype('float32'),np.asarray(y_train).astype('float32'), np.asarray(y_test).astype('float32')

In [ ]:
load_mouse_data()

In [3]:
import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import keras
from keras.models import Sequential
from keras.layers import Dense , Activation, Dropout
from keras.optimizers import Adam
# import adam optimizer

X_train, X_test, y_train, y_test = load_mouse_data()
input_shape = (X_train.shape[1],)
# Create the model
model = Sequential()
num_labels = 39
hidden_units = 256
batch_size = 128
dropout = 0.45

# Add layers to the model
model = Sequential()
model.add(Dense(64, input_dim=5, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(40, activation='softmax'))
# Compile the model
optimizer =Adam(learning_rate=0.001)
model.compile(optimizer = optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)


Epoch 1/10
6003/6003 [==============================] - 8s 1ms/step - loss: 0.0406 - accuracy: 0.6213 - val_loss: 0.0286 - val_accuracy: 0.6668
Epoch 2/10
 710/6003 [==>...........................] - ETA: 5s - loss: 0.0286 - accuracy: 0.6633

KeyboardInterrupt: 

In [ ]:
X_train =  sy.ActionObject.from_obj(X_train)
y_train =  sy.ActionObject.from_obj(y_train)
X_test =  sy.ActionObject.from_obj(X_test)
y_test =  sy.ActionObject.from_obj(y_test)
X_train_domain_obj = domain_client.api.services.action.set(X_train)
y_train_domain_obj = domain_client.api.services.action.set(y_train)
X_test_domain_obj = domain_client.api.services.action.set(X_test)
y_test_domain_obj = domain_client.api.services.action.set(y_test)


In [ ]:
@sy.syft_function(input_policy=sy.ExactMatch(X_train=X_train_domain_obj.id,y_train=y_train_domain_obj.id,X_test=X_test,y_train_domain_obj.id,y_test=y_test_domain_obj.id),
                  output_policy=sy.SingleExecutionExactOutput())
def train(X_train,y_train,X_test,y_test):
 import keras
 from keras.models import Sequential
 from keras.layers import Dense
 from keras.optimizers import Adam
 # import adam optimizer

 X_train, X_test, y_train, y_test = load_mouse_data()
 input_shape = (X_train.shape[1],)
 # Create the model
 model = Sequential()

 # Add layers to the model
 model.add(Dense(64, activation='relu', input_shape=input_shape))
 model.add(Dense(32, activation='relu'))
 model.add(Dense(1, activation='sigmoid'))

 # Compile the model
 optimizer =Adam(learning_rate=0.001)
 model.compile(optimizer = optimizer, loss='binary_crossentropy', metrics=['accuracy'])

 # Train the model
 model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

 # Evaluate the model
 loss, accuracy = model.evaluate(X_test, y_test)
 return loss,accuracy

In [ ]:
request = domain_client.code.request_code_execution(train)
request

In [ ]:
request.approve()


In [ ]:
result_ptr = domain_client.code.train_mlp(X_train=X_train.id,y_train=y_train.id,X_test=X_test.id,y_test=y_test.id)

In [ ]:
result_ptr.get()